# Prepare data for fine-tuning
Prepare data for fine-tuning the Open Grounding DINO model.

### 0. Import libraries and load data

In [1]:
import os
import sys
import json
import shutil

RAW_DATA_PATH = "../../data/raw/"
ANNOTATIONS_PATH = "../../data/annotations/"
STORAGE_PATH = "../../data/fine-tuning/baseline/"
sys.path.append("../annotate_dataset/")

from annotate_paintings_utils import *

In [2]:
filenames = ["annotations_10000_10009", "annotations_30_39", "annotations_100_109", "annotations_1000_1009", "annotations_2400_2409", "annotations_2500_2509", "annotations_7000_7009"]
all_annotations = []

for filename in filenames:
    with open(f"{ANNOTATIONS_PATH}iterative_grounding/{filename}.json") as f:
        all_annotations.extend(json.load(f)["annotations"])

### 1. Create data for fine-tuning in jsonl format

In [3]:
if not os.path.exists(STORAGE_PATH):
    os.mkdir(STORAGE_PATH)
    os.mkdir(STORAGE_PATH + "train/")
    os.mkdir(STORAGE_PATH + "val/")

In [4]:
def create_dataset(all_annotations, set_name):
    annotations = []

    for painting_annotations in all_annotations:
        painting_id = painting_annotations["painting_id"]
        extracted_objects = list(painting_annotations["objects"].keys())
        bboxes = painting_annotations["bounding_boxes"]
        image = load_image(painting_id)[1]

        current_annotation = {"filename": f"{painting_id}.png", "height": image.size[1], "width": image.size[0]}
        regions = []
        unique_labels = set()

        for bbox in bboxes:
            if bbox[0] not in extracted_objects:
                continue

            regions.append({"bbox": bbox[2], "phrase": bbox[0]})
            unique_labels.add(bbox[0])

        current_annotation["grounding"] = {"caption": " . ".join(unique_labels) + " .", "regions": regions} 

        if len(unique_labels) != 0:
            annotations.append(current_annotation)
            source_path = f"{RAW_DATA_PATH}filtered_paintings/{painting_id}.png"
            destination_path = f"{STORAGE_PATH}{set_name}/{painting_id}.png"
            shutil.copy2(source_path, destination_path)

    with open(f"{STORAGE_PATH}{set_name}/{set_name}_annotations.jsonl", "w", encoding='utf-8') as f:
        for item in annotations:
            json_line = json.dumps(item, ensure_ascii=False)
            f.write(json_line + '\n')

In [5]:
train_annotations = all_annotations[:len(all_annotations) // 2]
create_dataset(train_annotations, "train")
val_annotations = all_annotations[len(all_annotations) // 2:]
create_dataset(val_annotations, "val")